In [24]:
!pip install requests
!pip install python-dotenv
import json
import requests
import base64
import os
import sys
import dotenv

# can you see me write this
# yes :)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [25]:
## Using URL
## Functioning, but still only reading off the URL's

env_path = ".env"
proxy_key_name = "proxy_key"

system_message = "your task is to take an image and provide an alt text for it. keep the text short and to the point while still including all relevent detail. Don't use frilly words. Please begin the sentence with the format ALT ID:"
log_params = {}
image_url = "https://www.rorytm.com/pas4ai/nasa_blue_marble.jpg"


PROXY_KEY = ""

if "google.colab" in sys.modules:
   # we're in google colab so retrieve the proxy key from there
   from google.colab import userdata
   PROXY_KEY = userdata.get(proxy_key_name)
else:
  # retrieve the proxy key from local env file
  dotenv.load_dotenv(env_path)
  PROXY_KEY = os.getenv(proxy_key_name)


def openai_response(system, image_url, log_params, model="gpt-4o"):
    url = "https://pas4ai.rorytm.com:8001/proxy/openai"
    body = {
        "max_tokens": 300,
        "stream": False,
        "model": model,
        "temperature": 1,
        "presence_penalty": 0,
        "top_p": 1,
        "frequency_penalty": 0,
        "messages": [{
            "role": "system",
            "content": system,
        },{
            "role": "user",
            "content": [
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"{image_url}",
                  },
                },
              ],
            }
          ],
        "log_params": log_params,
    }
    headers = {"Authorization": f"Bearer {PROXY_KEY}"}
    response = requests.post(url, headers=headers, json=body, timeout=None)
    response_body = response.content.decode("utf-8")
    response_json = json.loads(response_body)["choices"][0]["message"]["content"]
    return response_json



response = openai_response(system_message, image_url, log_params)


print(response)
print(image_url)


# response = {
#     "choices": [
#         {
#             "message": {
#                 "role": "assistant",
#                 "content": [
#                     {"type": "text", "text": "What’s in this image?"},
#                     {
#                         "type": "image_url",
#                         "image_url": {
#                             "url": "https://www.rorytm.com/pas4ai/nasa_blue_marble.jpg",
#                         },
#                     },
#                 ],
#             }
#         }
#     ]
# }

ALT ID: A black and white speckled dog is lying on a wooden floor, peeking around a red wall while holding a tan plush toy in its mouth. Stonework and a dark hallway are visible in the background.
https://www.rorytm.com/pas4ai/nasa_blue_marble.jpg


In [26]:
## Decoding Base64
## Functioning, but still just making stuff up.


system_message = "your task is to take an image and provide an alt text for it. keep the text short and to the point while still including all relevent detail. Don't use frilly words. Please begin the sentence with the format ALT ID:"
log_params = {}



def openai_response(system, image_url, log_params, model="gpt-4o"):
    url = "https://pas4ai.rorytm.com:8001/proxy/openai"
    body = {
        "max_tokens": 300,
        "stream": False,
        "model": model,
        "temperature": 1,
        "presence_penalty": 0,
        "top_p": 1,
        "frequency_penalty": 0,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": f"Here is your image: {image_url}"}
        ],
        "log_params": log_params,
    }
    headers = {"Authorization": f"Bearer {PROXY_KEY}"}
    response = requests.post(url, headers=headers, json=body, timeout=None)
    response_body = response.content.decode("utf-8")
    response_json = json.loads(response_body)["choices"][0]["message"]["content"]
    return response_json






def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

image_path = "//nasa_blue_marble.jpg"

base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {PROXY_KEY}"
}

payload = {
    "model": "gpt-4o",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What’s in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens": 300
}


client = openai_response(system_message, image_url, log_params)
print(client)

FileNotFoundError: [Errno 2] No such file or directory: '//nasa_blue_marble.jpg'